# AutoCorrect Tool


### Motive: Whenever a word is typed we need to check the word and auto correct it if it's mispelled. 

##### HOW?
###### Approach 1
We can make use of **"Edit Distance"**.
"Edit distance" is a string metric, i.e. a way of quantifying how dissimilar two strings (e.g., words) are to one another, that is measured by counting the minimum number of operations required to transform one string into the other.
We can create a group of correct words(dictionary) and when the test word finds the distances from all the words in dictionary and replaces it with the word having minimum distance. 

We will make use of **Levenshtein distance**.
Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.

In [3]:
#Function to calculate Levenshtein distance between two words
def editDistance(s: str, t: str) -> int:
    n = len(s)
    m = len(t)

    prev = [j for j in range(m+1)]
    curr = [0] * (m+1)

    for i in range(1, n+1):
        curr[0] = i
        for j in range(1, m+1):
            if s[i-1] == t[j-1]:
                curr[j] = prev[j-1]
            else:
                mn = min(1 + prev[j], 1 + curr[j-1])
                curr[j] = min(mn, 1 + prev[j-1])
        prev = curr.copy()

    return prev[m]


In [4]:
#Creating a list calculating the word's distance from every other words in the dictionary
def find_word(diction: list,word: str) -> list:
    final=[]
    for w in diction:
        final.append(editDistance(w,word))
    return final

In [5]:
diction = ["save","happy","I","want","realistic","this","intuitive","from","to","am","rage","spell","check","fortune","resonance","spelling","country","Please","is","a"]
text = "I am hapu to sve ths county from rage spleling"
text2=text.split(" ");
ans=""
for word in text2:
    dist_word = find_word(diction,word)
    fword = diction[dist_word.index(min(dist_word))]
    ans = ans+fword+" "
print("Text Before: "+text)
print("Text After: "+ans)


Text Before: I am hapu to sve ths county from rage spleling
Text After: I am happy to save this country from rage spelling 


BUt this approach has many limitations in real world:
(1) **Takes much time and space**
Time Complexity: O(len(word) x len(dictionary))
Space Complexity: O(len(Dictionary))
And dictionary needs to be huge to encompass all the english words.
(2) **Many words may have same minimum distance and so it becomes to difficult to choose which word is optimal to choose**
(3) **It also becomes difficult to choose words in which context is the sentence being written**
There can be chance that the desired word was not one with the minimum Levenshtein distance.

##### Approach 2
Now the previous approach gave us some lead to find the close words which the user might want to type, but which word he/she wants, this can be done by seeing their previous words which has been written and make a probabilistic assumptions from that probability list. This is an excellent application of A.I. specifically learning from the past experiences.

Before moving to more further implementation, we would see another sort of string comparing method known as jacard index, it measures the similarity between two words and this can be helpful as to predict the word here. A very useful library text distance has many of this string matching algorithm, thus we can make use of it.

In [9]:
%pip install textdistance
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pandas-2.0.3-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import textdistance
import numpy as np
import pandas as pd
from collections import Counter
import re

ModuleNotFoundError: No module named 'pandas'

In [1]:
words=[]
with open('rtext.txt','r',encoding="utf8") as f:
    fd = f.read()
    fd=fd.lower()
    words = re.findall('\w+',fd)
# Dictionary(Vocabulary) of words present in text
D = set(words)
print(f"Words in the text:{words[0:10]}")
print(f"Total Unique words are {len(D)}.")
    

NameError: name 're' is not defined

In [12]:
word_freq = {}  
word_freq = Counter(words)
print(word_freq.most_common()[0:10])

[('the', 14703), ('of', 6742), ('and', 6517), ('a', 4799), ('to', 4707), ('in', 4238), ('that', 3081), ('it', 2534), ('his', 2530), ('i', 2120)]


In [13]:
probs = {}     
Total = sum(word_freq.values())    
for k in word_freq.keys():
    probs[k] = word_freq[k]/Total

In [23]:
def my_autocorrect(input_word):
    input_word = input_word.lower()
    if input_word in D:
        return('Your word seems to be correct')
    else:
        sim = [1-(textdistance.Jaccard(qval=1).distance(v,input_word)) for v in word_freq.keys()]
        df = pd.DataFrame.from_dict(probs, orient='index').reset_index()
        df = df.rename(columns={'index':'Word', 0:'Prob'})
        df['Similarity'] = sim
        output = df.sort_values(['Similarity', 'Prob'], ascending=False).head()
        return(output)

In [27]:
my_autocorrect("spll")

,Word,Prob,Similarity
3592,spell,0.000049,0.800000
13391,pills,0.000009,0.800000
16791,spill,0.000004,0.800000
16963,pulls,0.000004,0.800000
7958,spells,0.000004,0.666667
